In [1]:
from transformer_lens import HookedTransformer, ActivationCache
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch as t
from AttenScope import AttentionPostprocessing, TensorToHeatmap
import numpy

device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

/home/duncan/anaconda3/envs/Synstiller/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Python-hf")
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Python-hf", torch_dtype=t.float16
)

/home/duncan/anaconda3/envs/Synstiller/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.59it/s]


In [3]:
hooked_model = HookedTransformer.from_pretrained_no_processing(
    model_name="CodeLlama-7b-Python-hf",
    hf_model=model,
    tokenizer=tokenizer,
    device=device,
    dtype="float16",
)

Loaded pretrained model CodeLlama-7b-Python-hf into HookedTransformer


In [9]:
testcode = """
def fibo(n):
    if n <= 1:
        return n
    else:
        return fibo(n - 1) + fibo(n - 2)
"""

In [10]:
try:
    del (
        logits,
        cache,
        toks,
    )
except:
    pass
toks = hooked_model.to_tokens(testcode)
logits, cache = hooked_model.run_with_cache(toks)
str_toks = hooked_model.to_str_tokens(toks)

In [51]:
attentions = cache["pattern", 0][0].to("cpu")
values = cache["v", 0][0].to("cpu").transpose(0, 1)
attentions = [
    AttentionPostprocessing.apply_value_norm_to_attention(a, v).numpy()
    for a, v in zip(attentions, values)
]

labels = [f"head {i}" for i in range(len(attentions))]
html = TensorToHeatmap.generate_heatmap_html(attentions, labels, str_toks)
with open("atten-scope-test.html", "w") as f:
    f.write(html)
print("Save at atten-scope-test.html")

Save at /var/www/html/duncan/atten-scope-test.html
